In [94]:
import pandas as pd   
from matplotlib import pyplot as plt
from numpy import mean
import time

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold

import warnings 
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression

In [95]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [ ]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [ ]:
#from df2 drop sub_area and row ID 
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [ ]:
df2 = df2.drop(columns=[ 'row ID'])

In [96]:
category_frequencies = df1['sub_area'].value_counts()

print(category_frequencies)

Poselenie Shhapovskoe             1105
Alekseevskoe                      1104
Tverskoe                          1101
Poselenie Mosrentgen              1099
Caricyno                          1097
                                  ... 
Posleenie Novofedorovskoe            3
Poseleine Voronovskoe                3
Poselenie Mhiajlovo-Jarcevskoe       3
Prsopekt Vernadskogo                 2
Poselenie Krasnopahorksoe            2
Name: sub_area, Length: 1927, dtype: int64


In [97]:
infrequent_categories = category_frequencies[category_frequencies < 900].index.tolist()
print(len(infrequent_categories))
print(len(category_frequencies))

1781
1927


In [98]:
df1['sub_area'] = df1['sub_area'].replace(infrequent_categories, 'Other')
df2['sub_area'] = df2['sub_area'].replace(infrequent_categories, 'Other')
category_frequencies1 = df1['sub_area'].value_counts()
print(len(category_frequencies1))

147


In [99]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [ ]:
print

In [100]:
X = df_onehot.drop(columns=['price_doc'])
print(X.shape)
y = df_onehot[['price_doc']]






(181507, 434)


In [101]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(test_onehot)

In [102]:
#drop row id from x_testscaled
X_testscaled = X_testscaled[:, 1:]


In [ ]:
print(X_testscaled.shape)

In [103]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(X_testscaled)


In [ ]:
#give me code for feature importance using linear regression
model = LinearRegression()
model.fit(X_scaled, y)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# drop features with low importance
threshold = 100  # Adjust the threshold as needed
selected_features = [feature for feature, score in zip(X.columns, importance) if score >= threshold]
X_selected = X[selected_features]

# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()



In [ ]:
from sklearn.decomposition import PCA
n_components = 150  # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(X_scaled)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(X_testscaled)

In [104]:
# original_feature_indices = np.arange(X.shape[1])
threshold_value = 0.001
variance_filter = VarianceThreshold(threshold=threshold_value)

# Fit and transform the feature matrix
X_filtered = variance_filter.fit_transform(X_scaled)

X_test_high_variance = variance_filter.transform(X_testscaled)

In [ ]:
print(X_selected.shape)

In [105]:
print(X_filtered.shape)

(181507, 434)


In [ ]:
print(X_test_high_variance.shape)

In [ ]:
#store x_filtered as a df
X_filtered = pd.DataFrame(X_filtered)

In [ ]:
#keep only the selected features in the testone hot
test_onehot = X_testscaled[X_selected.columns]
test_onehot.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_selected)
test_scaled = scaler.transform(test_onehot)
print(test_scaled.shape)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print("Training Set Info:")
print(X.info())

print("\nTest Set Info:")
print(test_onehot.info())


In [106]:

#print("Mean squared error: %.2f" % mean_squared_error(test_onehot, md_probs))

0.6408917080858367
[[ 1.04796802e+04  2.25970254e+02  1.87332927e+05  9.61639602e+04
   2.90857903e+05  1.49098190e+05  6.17123331e+04  2.88755942e+05
   1.27101165e+05 -1.37689517e+05 -2.37691860e+05  1.75653281e+05
   3.57977687e+05 -6.29029836e+04  1.72839858e+04 -9.37082685e+03
   3.23600828e+04  2.60174997e+05  5.81995297e+04 -1.33596050e+04
   8.99734588e+04  2.19186603e+04  3.03045339e+04 -6.90118899e+04
   2.09299001e+05  7.22331891e+04 -1.53541241e+05 -6.76935664e+04
  -1.69977598e+05 -9.79656835e+04  1.22098072e+05 -1.03062693e+05
  -4.59790631e+04  2.30793912e+05 -3.81214891e+04  5.83814834e+04
  -1.00389781e+05  7.20524340e+04  4.29269488e+04 -1.67215205e+05
   2.17864034e+04 -5.72710438e+04  3.47023897e+04 -8.56263750e+04
  -2.10722863e+05 -2.15805489e+05  7.67337511e+04 -2.95513418e+04
   4.42157720e+04 -1.11830615e+05  1.14808618e+05 -6.24762530e+03
   1.35831072e+05 -1.23809421e+05  9.00110925e+04  5.64859864e+04
  -1.47516117e+05  1.18211813e+05  6.22084471e+04 -5.0070

In [107]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions15.csv', index=False)